<a href="https://colab.research.google.com/github/HYORIMLEE/KaggleKiggle/blob/main/jwjw/kakao_arena/melon/my/melon_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
# 구 버전 jupyter에서의 그래프 시각화

plt.rc('font',family='NanumGothic')

plt.rc("axes",unicode_minus=False)
# 마이너스 폰트 깨짐 방지

%config InlineBackend.figure_format = 'retina'
# 폰트를 선명하게

In [55]:
genre_gn_all = pd.read_json('genre_gn_all.json',typ='series')

In [56]:
# make genre_gn_all as data frame with code, name columns
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns={'index' : 'gnr_code'})

In [57]:
# 254 categories
genre_gn_all

,gnr_code,gnr_name
0,GN0100,발라드
1,GN0101,세부장르전체
2,GN0102,'80
3,GN0103,'90
4,GN0104,'00
...,...,...
249,GN2900,뮤지컬
250,GN2901,세부장르전체
251,GN2902,국내뮤지컬
252,GN2903,국외뮤지컬


In [ ]:
# filter tail of code is '00', which is a big category
gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] == '00']

In [ ]:
# 30 big categories
gnr_code

In [ ]:
# filter tail of code is not '00', which is a detail category
dtl_gnr_code = genre_gn_all[genre_gn_all['gnr_code'].str[-2:] != '00']
dtl_gnr_code.rename(columns = {'gnr_code' : 'dtl_gnr_code', 'gnr_name' : 'dtl_gnr_name'}, inplace = True)

In [ ]:
# 224 detail categories
dtl_gnr_code

In [ ]:
# extract 4 head char of code => genre code
gnr_code = gnr_code.assign(join_code = gnr_code['gnr_code'].str[0:4])
dtl_gnr_code = dtl_gnr_code.assign(join_code = dtl_gnr_code['dtl_gnr_code'].str[0:4])

# make it tree by extracted genre code
gnr_code_tree = pd.merge(gnr_code,dtl_gnr_code, how = 'left', on = 'join_code')


In [ ]:
gnr_code_tree

In [ ]:
# see code GN0900 POP
gnr_code_tree[gnr_code_tree['gnr_code'] == 'GN0900']

In [ ]:
song_meta = pd.read_json('song_meta.json',typ = 'frame')

In [ ]:
song_meta

In [ ]:
song_meta.info()

In [ ]:
# extract id, big genre list
song_gnr_map = song_meta.loc[:,["id","song_gn_gnr_basket"]]

# match gen code and id 1 by 1 as array
song_gnr_map_unnest = np.dstack(
  (
    # extract id, gen list length as array
    # and repeat id as value of list
    np.repeat(song_gnr_map.id.values, list(map(len, song_gnr_map.song_gn_gnr_basket))),
    
    # get gen lists as array and merge as one list (duplicate allowed)
    np.concatenate(song_gnr_map.song_gn_gnr_basket.values)
  )
)

In [ ]:
# make it dataframe
song_gnr_map = pd.DataFrame(data = song_gnr_map_unnest[0], columns = song_gnr_map.columns)

# change column names
song_gnr_map['id'] = song_gnr_map['id'].astype(str)
song_gnr_map.rename(columns = {'id' : 'song_id', 'song_gn_gnr_basket' : 'gnr_code'}, inplace = True)

# remove unnest
del song_gnr_map_unnest

In [ ]:
song_gnr_count = song_gnr_map.groupby('song_id').gnr_code.nunique().reset_index(name = 'mapping_gnr_cnt')
song_gnr_count

In [ ]:
np.concatenate(song_gnr_map.song_gn_gnr_basket.values)